In [ ]:
# !pip install imbalanced-learn
!pip install -U skorch
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.8 MB/s eta 0:00:00


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import torch
from torch import nn
import skorch
from skorch import NeuralNetClassifier
from sklearn.metrics import classification_report
from skorch.callbacks import EarlyStopping
from skorch.helper import predefined_split
import optuna
from sklearn.metrics import f1_score

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Used to split the dataset into train, validation, and test sets based on time
def assign_group(time):
    """
    Assigns a group label ('train', 'val', or 'test') based on a timestamp.

    Parameters:
        time (pd.Timestamp): A timestamp to evaluate.

    Returns:
        str: Group label ('train', 'val', or 'test').
    """
    if time < pd.Timestamp('2023-01-01'):
        return 'train'
    elif pd.Timestamp('2023-01-01') <= time < pd.Timestamp('2023-09-01'):
        return 'val'
    else:
        return 'test'

# Preprocess data
def data_preprocess(df, locations):
    """
    Prepares and preprocesses a DataFrame by filtering locations, computing velocity
    and acceleration, assigning groups, and removing NaN values.

    Parameters:
        df (pd.DataFrame): The input data containing 'Time', 'Location', and 'Temperature'.
        locations (list): List of locations to include in the processing.

    Returns:
        pd.DataFrame: A DataFrame with 'Time', 'Location', 'group', 'Temperature',
                      'Velocity', and 'Acceleration' columns.
    """
    df['Time'] = pd.to_datetime(df['Time'])
    df = df[df['Location'].isin(locations)]

    # Sort the DataFrame by 'Location' and 'Time'
    df = df.sort_values(by=['Location', 'Time']).reset_index(drop=True)

    # Compute Velocity and Acceleration
    df['Velocity'] = df.groupby('Location')['Temperature'].diff() / (15 * 60)
    df['Acceleration'] = df.groupby('Location')['Velocity'].diff() / (15 * 60)

    # Assign groups (train, val, test)
    df['group'] = df['Time'].apply(assign_group)

    # Drop rows with NaN values in Velocity or Acceleration
    df = df.dropna(subset=['Velocity', 'Acceleration']).reset_index(drop=True)

    return df[['Time', 'Location', 'group', 'Temperature', 'Velocity', 'Acceleration']]

# Create Rolling Windows
def create_rolling_windows(arr, window_size=16, horizon=8):
    """
    Creates rolling windows of features and corresponding targets for time-series data.

    Parameters:
        arr (np.ndarray): Input array of shape (n_samples, n_features).
        window_size (int): Number of consecutive timesteps in a window (default=16).
        horizon (int): horizon between the end of the window and the target value (default=8).

    Returns:
        tuple: Two numpy arrays:
               - X: Features of shape (n_windows, window_size, n_features)
               - y: Target values of shape (n_windows,)
    """
    X, y = [], []
    for i in range(len(arr) - window_size - horizon):
        X.append(arr[i:i + window_size])  # Features
        y.append(arr[i + window_size + horizon, 0])  # Target
    return np.array(X), np.array(y)

def create_feature_matrix(df, window_size=16, horizon=8):
    """
    Creates a feature matrix and target vector using rolling windows for time-series data.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Temperature', 'Velocity', and 'Acceleration'.
        window_size (int): Number of consecutive timesteps in a window (default=16).
        horizon (int): horizon between the end of the window and the target value (default=8).

    Returns:
        tuple: Two numpy arrays:
               - X: Features of shape (n_windows, window_size, n_features)
               - y: Target values of shape (n_windows,)
    """
    # Order matters: Temperature, Velocity, Acceleration.
    feature_matrix = np.column_stack((df['Temperature'].values,
                                      df['Velocity'].values,
                                      df['Acceleration'].values))
    X_matrix, y_vector = create_rolling_windows(feature_matrix, window_size, horizon)
    return X_matrix, y_vector

def prepare_datasets(df, Config):
    """
    Prepares training datasets by creating rolling windows for multiple locations.

    Parameters:
        df (pd.DataFrame): The input data.
        locations (list): List of locations to process.
        window_size (int): Number of consecutive timesteps in a window (default=16).
        horizon (int): horizon between the end of the window and the target value (default=8).

    Returns:
        tuple: Two numpy arrays:
               - X_all: Combined feature arrays of shape (total_windows, window_size, n_features).
               - y_all: Combined target arrays of shape (total_windows,).
    """

    X_all, y_all = [], []
    locations = Config.locations
    window_size = Config.window_size
    horizon = Config.horizon

    for location in locations:
        dff = df[df['Location'] == location]
        X, y = create_feature_matrix(dff, window_size, horizon)
        if len(X) > 0 and len(y) > 0:
            X_all.append(X)
            y_all.append(y)

    X_all = np.concatenate(X_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)

    return X_all, y_all

def add_augmented_data(X, y):
    """
    Augments the dataset by duplicating and slightly modifying samples
    where the target value is 1.0 to address class imbalance.

    Parameters:
        X (np.ndarray): Feature array of shape (n_samples, window_size, n_features).
        y (np.ndarray): Target array of shape (n_samples,).

    Returns:
        tuple: Augmented feature array and target array.
    """
    duplicates_idx = np.where(y == 1.0)[0]  # Find indices where target is 1.0
    X_copy, y_copy = np.copy(X), np.copy(y)
    while True:
        # Add random noise to duplicate samples
        X_duplicated = X_copy[duplicates_idx] + 0.01 * np.random.randn(*X[duplicates_idx].shape)
        y_duplicated = y_copy[duplicates_idx]
        X = np.concatenate([X, X_duplicated], axis=0)
        y = np.concatenate([y, y_duplicated], axis=0)

        # Stop augmenting if the target ratio exceeds 4%
        if np.mean(y) > 0.04:
            print(f"Augmentation completed. {np.mean(y) * 100:.2f}% of train data is 1.0")
            break
    return X, y

# Define Model
class GRUNet(nn.Module):
    """
    A Gated Recurrent Unit (GRU) neural network for time-series forecasting.

    Parameters:
        hidden_size (int): Number of hidden units in each GRU layer (default=16).
        num_layers (int): Number of GRU layers (default=2)
    """
    def __init__(self, hidden_size=16, num_layers=2, dropout=0.5):
        super(GRUNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(3, hidden_size, num_layers, batch_first=True,
                          dropout=dropout if num_layers > 1 else 0.0)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Forward pass of the GRU network.

        Parameters:
            x (torch.Tensor): Input tensor of shape (batch_size, sequence_length, input_size).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, output_size).
        """
        out, _ = self.gru(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out


In [ ]:
class Config:
    """
    Static configuration class to store imporant parameters.
    """
    window_size = 16
    horizon = 8
    threshold = 85.0
    batch_size = 64
    # air-conditioned cooling system
    locations = ['C76', 'C80', 'E20', 'E30', 'E39', 'L18', 'L20', 'S44']
    # forced cooling system
    # locations = ["A20", "A30", "A40", "A60", "A70", "A80",
    #              "K20", "K30", "M10", "M50", "M90", "R10",
    #              "R20", "R30", "R40", "R50", "W10"]

In [ ]:
# Load dataset
# dataframe should have ['Time', 'Location', 'Temperature', 'forced_constant_ventilation_cooling'(optional)] columns
df = pd.read_csv('/content/drive/MyDrive/BART/data/cleaned_data_v3.csv')
df.head()

,Time,Location,Temperature,forced_constant_ventilation_cooling
0,2020-12-31 23:50:01,A20,59.9,1
1,2020-12-31 23:35:01,A20,59.6,1
2,2020-12-31 23:20:01,A20,60.3,1
3,2020-12-31 23:05:01,A20,60.6,1
4,2020-12-31 22:50:01,A20,60.4,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6175997 entries, 0 to 6175996
Data columns (total 4 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   Time                                 object 
 1   Location                             object 
 2   Temperature                          float64
 3   forced_constant_ventilation_cooling  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 188.5+ MB


In [ ]:
df.shape

(6175997, 4)

In [ ]:
# preprocess
# data_preprocess return ['Time', 'Location', 'group', 'Temperature', 'Velocity', 'Acceleration'] columns
df = data_preprocess(df, Config.locations)

# split dataframe into train/val/test
train_df = df[df['group'] == 'train']
val_df = df[df['group'] == 'val']
test_df = df[df['group'] == 'test']

# transform dataframe into X, y matrix, shape is (total_windows, window_size, n_features)
X_train, y_train = prepare_datasets(train_df, Config)
X_val, y_val = prepare_datasets(val_df, Config)
X_test, y_test = prepare_datasets(test_df, Config)

# standardize train, val, test
train_mean = np.mean(X_train, axis=(0, 1))
train_std = np.std(X_train, axis=(0, 1))
X_train  = (X_train - train_mean) / train_std
X_val  = (X_val - train_mean) / train_std
X_test  = (X_test - train_mean) / train_std

# change overheat into 1.0 else 0.0
y_train = (y_train > Config.threshold).astype(float)
y_val = (y_val > Config.threshold).astype(float)
y_test = (y_test > Config.threshold).astype(float)

# add augmented data to x_train, y_train here
X_train, y_train = add_augmented_data(X_train, y_train)

# skorch expect float32 as datatype of numpy/torch array so make sure data type is float 32
X_train, y_train = X_train.astype(np.float32), y_train.astype(np.float32).reshape(-1, 1)
X_val, y_val = X_val.astype(np.float32), y_val.astype(np.float32).reshape(-1, 1)
X_test, y_test = X_test.astype(np.float32), y_test.astype(np.float32).reshape(-1, 1)

# pre-defined validation set
valid_ds = skorch.dataset.Dataset(X_val, y_val)

Augmentation completed. 4.25% of train data is 1.0


## Hyperparameter Tuning

This section identifies the optimal model parameters (such as `hidden_size`, `layer_num`, and `dropout rate`) as well as the learning rate.

Here, I have selected only the most important parameters for tuning, but you can expand the hyperparameter search space as much as you need.


In [ ]:
# Define your objective function for Optuna

def objective(trial):

    hidden_size = trial.suggest_categorical('hidden_size', [16, 32, 64])
    num_layers = trial.suggest_categorical('num_layers', [1, 2])
    learning_rate = trial.suggest_float('lr', 1e-5, 1e-2)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    model = NeuralNetClassifier(
            GRUNet,
            module__hidden_size=hidden_size,
            module__num_layers=num_layers,
            module__dropout=dropout,
            # smaller epochs are enough to evaluate
            max_epochs=5,
            criterion=nn.BCEWithLogitsLoss,
            optimizer=torch.optim.AdamW,
            lr=learning_rate,
            # no validation dataset is needed here because this validation dataset is supposed to be used internally to use early stopping
            # but we don't need early stopping when tuning hyperparams, thus we set None to train_split
            train_split = None,
            batch_size=Config.batch_size,
            iterator_train__shuffle=True,
            verbose=1,
        )

    model = model.fit(X_train, y_train)
    predicted = model.predict(X_val)
    y_pred = predicted.flatten()
    y_true = y_val.flatten()
    score = f1_score(y_true, y_pred, pos_label=1)
    return score

# Perform optimization using Optuna, it maximizes f1-score
study = optuna.create_study(direction="maximize")

# change n_trials depends on how much time you would spend for param tuning
study.optimize(objective, n_trials=30)

# Print the best parameters
print("Best parameters: ", study.best_params)

# store study.best_params to best_params
best_params = study.best_params

[I 2024-12-08 07:44:36,230] A new study created in memory with name: no-name-a715700c-d2b5-4f3e-8c97-df7fd5974058


  epoch    train_loss      dur
-------  ------------  -------
      1        0.0498  59.9564
      2        0.0422  60.6858
      3        0.0390  60.8210
      4        0.0365  60.9378
      5        0.0343  60.5939


[I 2024-12-08 07:49:47,680] Trial 0 finished with value: 0.6216867469879518 and parameters: {'hidden_size': 64, 'num_layers': 1, 'lr': 0.0011999050670054394, 'dropout': 0.2757141144342244}. Best is trial 0 with value: 0.6216867469879518.


  epoch    train_loss      dur
-------  ------------  -------
      1        0.0496  80.6326
      2        0.0421  79.5872
      3        0.0392  81.0314
      4        0.0376  81.1054
      5        0.0366  81.8614


[I 2024-12-08 07:56:39,550] Trial 1 finished with value: 0.5476419634263715 and parameters: {'hidden_size': 16, 'num_layers': 2, 'lr': 0.0017303730930262576, 'dropout': 0.06735818802560317}. Best is trial 0 with value: 0.6216867469879518.


...

  epoch    train_loss      dur
-------  ------------  -------
      1        0.0467  89.5670
      2        0.0394  87.8240
      3        0.0365  88.6769
      4        0.0339  86.9646
      5        0.0320  86.2390


[I 2024-12-08 10:37:07,011] Trial 28 finished with value: 0.5939947780678851 and parameters: {'hidden_size': 32, 'num_layers': 2, 'lr': 0.0029422516366393414, 'dropout': 0.11658270002589122}. Best is trial 14 with value: 0.689041095890411.


  epoch    train_loss      dur
-------  ------------  -------
      1        0.0537  45.4526
      2        0.0460  44.9162
      3        0.0446  45.7208
      4        0.0436  46.0001
      5        0.0427  45.6066


[I 2024-12-08 10:41:00,184] Trial 29 finished with value: 0.6105383734249714 and parameters: {'hidden_size': 16, 'num_layers': 1, 'lr': 0.002212115428327964, 'dropout': 0.31149971187307346}. Best is trial 14 with value: 0.689041095890411.


Best parameters:  {'hidden_size': 32, 'num_layers': 1, 'lr': 0.007462825116974809, 'dropout': 0.4180932596837986}


In [ ]:
best_params

{'hidden_size': 32,
 'num_layers': 1,
 'lr': 0.007462825116974809,
 'dropout': 0.4180932596837986}

## OOS validation

After completing the hyperparameter tuning, we now know the optimal parameters to use, and these have been stored in the best_params dictionary.

In [ ]:
best_model = NeuralNetClassifier(
    GRUNet,
    module__hidden_size=best_params['hidden_size'],
    module__num_layers=best_params['num_layers'],
    module__dropout=best_params['dropout'],
    max_epochs=30,
    criterion=nn.BCEWithLogitsLoss,
    optimizer=torch.optim.AdamW,
    lr=best_params['lr'],
    batch_size=Config.batch_size,
    # this is early stopping, it stops training iterations if validation loss does not improve for 5 epochs
    callbacks=[EarlyStopping(monitor='valid_loss',
                           patience=5,
                           load_best=True)],
    iterator_train__shuffle=True,
    # use valid_ds as validation dataset to calculate validation loss which is used for early stopping.
    train_split=predefined_split(valid_ds),
    verbose=1,
)

# net add val dataset
best_model.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.0495       0.9973        0.0116  58.8334
      2        0.0455       0.9970        0.0101  60.0703
      3        0.0445       0.9935        0.0170  58.0195
      4        0.0434       0.9959        0.0111  60.6419
      5        0.0430       0.9970        0.0098  58.2511
      6        0.0426       0.9967        0.0103  58.2111
      7        0.0423       0.9967        0.0115  57.7560
      8        0.0423       0.9969        0.0100  57.9812
      9        0.0421       0.9975        0.0096  58.4369
     10        0.0415       0.9969        0.0101  57.7782
     11        0.0418       0.9963        0.0095  57.2982
     12        0.0418       0.9952        0.0150  57.5152
     13        0.0415       0.9971        0.0097  57.8104
     14        0.0416       0.9975        0.0094  57.0045
     15        0.0416       0.9973        0.0099  57.1321
     16       

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=GRUNet(
    (gru): GRU(3, 32, batch_first=True)
    (dropout): Dropout(p=0.4180932596837986, inplace=False)
    (fc): Linear(in_features=32, out_features=1, bias=True)
  ),
)

In [ ]:
predicted = best_model.predict(X_test)
y_pred = predicted.flatten()
y_true = y_test.flatten()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    186207
         1.0       0.76      0.83      0.79       167

    accuracy                           1.00    186374
   macro avg       0.88      0.92      0.90    186374
weighted avg       1.00      1.00      1.00    186374

